In [6]:
from pathlib import Path
import pandas as pd

# 조치원/고려대세종 근방 중심
CENTER_LAT = 36.6110
CENTER_LON = 127.2870

# 너가 이미 계산한 격자점 14개 (lat,lon,distance_km) — 이미 selected가 있다면 이 셀은 생략 가능
# 14개 격자점 (너가 출력한 그대로)
selected = pd.DataFrame({
    "lat": [
        36.50, 36.75, 36.50, 36.75, 36.50, 36.75, 36.25,
        36.50, 37.00, 36.75, 36.25, 37.00, 36.25, 36.50
    ],
    "lon": [
        127.25, 127.25, 127.50, 127.50, 127.00, 127.00, 127.25,
        127.75, 127.25, 127.75, 127.50, 127.50, 127.00, 126.75
    ],
    "distance_km": [
        12.777434, 15.804355, 22.678240, 24.488381, 28.451624, 29.898450, 40.277622,
        43.157987, 43.380079, 44.086556, 44.434882, 47.229158, 47.650846, 49.527714
    ]
})

# ERA5 CDS area format: [North, West, South, East]
lat_min, lat_max = selected["lat"].min(), selected["lat"].max()
lon_min, lon_max = selected["lon"].min(), selected["lon"].max()

# 버퍼(0.25도): 가장자리 잘림 방지 + 안정적으로 격자 포함
buffer_deg = 0.25
AREA = [
    float(lat_max + buffer_deg),  # North
    float(lon_min - buffer_deg),  # West
    float(lat_min - buffer_deg),  # South
    float(lon_max + buffer_deg),  # East
]

# ERA5 다운로드 변수 (권장)
VARIABLES = [
    "2m_temperature",
    "total_precipitation",
    "2m_dewpoint_temperature",
    "surface_pressure",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "surface_solar_radiation_downwards",
]

# 데이터 폴더
RAW_DIR = Path("data_raw")
UNZIP_DIR = RAW_DIR / "unzipped"
PROC_DIR = Path("data_processed")

RAW_DIR.mkdir(exist_ok=True)
UNZIP_DIR.mkdir(parents=True, exist_ok=True)
PROC_DIR.mkdir(exist_ok=True)

# AREA는 너가 이미 계산한 걸 그대로 사용 (없으면 아래처럼 만들면 됨)
# AREA = [N, W, S, E]
# 예: AREA = [37.25, 126.75, 36.25, 127.75]
# 14개 격자점 (너가 출력한 그대로)
selected = pd.DataFrame({
    "lat": [
        36.50, 36.75, 36.50, 36.75, 36.50, 36.75, 36.25,
        36.50, 37.00, 36.75, 36.25, 37.00, 36.25, 36.50
    ],
    "lon": [
        127.25, 127.25, 127.50, 127.50, 127.00, 127.00, 127.25,
        127.75, 127.25, 127.75, 127.50, 127.50, 127.00, 126.75
    ],
    "distance_km": [
        12.777434, 15.804355, 22.678240, 24.488381, 28.451624, 29.898450, 40.277622,
        43.157987, 43.380079, 44.086556, 44.434882, 47.229158, 47.650846, 49.527714
    ]
})

# ERA5 CDS area format: [North, West, South, East]
lat_min, lat_max = selected["lat"].min(), selected["lat"].max()
lon_min, lon_max = selected["lon"].min(), selected["lon"].max()

# 버퍼(0.25도): 가장자리 잘림 방지 + 안정적으로 격자 포함
buffer_deg = 0.25
AREA = [
    float(lat_max + buffer_deg),  # North
    float(lon_min - buffer_deg),  # West
    float(lat_min - buffer_deg),  # South
    float(lon_max + buffer_deg),  # East
]

print("selected shape:", selected.shape)
print("AREA [N, W, S, E]:", AREA)
selected.head()


selected shape: (14, 3)
AREA [N, W, S, E]: [37.25, 126.5, 36.0, 128.0]


,lat,lon,distance_km
0,36.50,127.25,12.777434
1,36.75,127.25,15.804355
2,36.50,127.50,22.678240
3,36.75,127.50,24.488381
4,36.50,127.00,28.451624


In [7]:
import zipfile
import xarray as xr

def open_era5_zip_or_nc(path: Path):
    """
    CDS에서 받은 파일이
    - ZIP(PK..)이면 압축 풀어 nc 여러개 merge
    - 진짜 nc면 그대로 open
    또한 valid_time -> time로 통일
    """
    path = Path(path)
    with open(path, "rb") as f:
        sig = f.read(2)

    if sig == b"PK":
        out_dir = UNZIP_DIR / path.stem
        out_dir.mkdir(parents=True, exist_ok=True)
        with zipfile.ZipFile(path, "r") as z:
            z.extractall(out_dir)
        nc_files = sorted(out_dir.glob("*.nc"))
        if not nc_files:
            raise ValueError(f"No .nc found in zip: {path}")
        dss = [xr.open_dataset(nc, engine="netcdf4") for nc in nc_files]
        # valid_time -> time 통일
        for i in range(len(dss)):
            if "valid_time" in dss[i].coords:
                dss[i] = dss[i].rename({"valid_time": "time"})
        ds = xr.merge(dss, compat="override", join="outer")
    else:
        ds = xr.open_dataset(path, engine="netcdf4")
        if "valid_time" in ds.coords:
            ds = ds.rename({"valid_time": "time"})

    if "time" not in ds.coords:
        raise ValueError(f"No time coordinate found. coords={list(ds.coords)}")

    ds = ds.sortby("time")
    return ds


In [8]:
import numpy as np

def ds_to_parquet(ds, out_parquet: Path):
    df = ds.to_dataframe().reset_index()

    # 좌표 통일
    if "latitude" in df.columns: df = df.rename(columns={"latitude":"lat"})
    if "longitude" in df.columns: df = df.rename(columns={"longitude":"lon"})

    # time 포함 체크
    if "time" not in df.columns:
        raise ValueError(f"time not in columns: {df.columns}")

    # Spark Parquet 호환을 위해 time을 ms 정밀도 string으로 저장
    df["time"] = pd.to_datetime(df["time"]).dt.floor("ms").astype(str)

    keep_candidates = ["time","lat","lon","t2m","tp","d2m","sp","u10","v10","ssrd"]
    keep = [c for c in keep_candidates if c in df.columns]
    df = df[keep].copy()

    out_parquet.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(out_parquet, index=False)
    return out_parquet, df.shape


In [9]:
import cdsapi

c = cdsapi.Client()

def download_era5_month(year: int, month: int, area, variables):
    y = f"{year:04d}"
    m = f"{month:02d}"

    # raw는 "확장자 신경쓰지 말고" bin으로 저장 (실제는 ZIP일 수 있음)
    raw_path = RAW_DIR / f"era5_hourly_{y}{m}_jochiwon.bin"
    pq_path  = PROC_DIR / f"era5_hourly_{y}{m}_jochiwon.parquet"

    if pq_path.exists():
        return "SKIP(parquet exists)", pq_path

    # 다운로드
    if not raw_path.exists():
        c.retrieve(
            "reanalysis-era5-single-levels",
            {
                "product_type": "reanalysis",
                "variable": variables,
                "year": y,
                "month": m,
                "day": [f"{d:02d}" for d in range(1, 32)],
                "time": [f"{h:02d}:00" for h in range(24)],
                "area": area,
                "format": "netcdf",
            },
            str(raw_path)
        )

    # 열기(Zip/NC 자동) → parquet 변환
    ds = open_era5_zip_or_nc(raw_path)
    out_pq, shape = ds_to_parquet(ds, pq_path)

    return f"OK {y}{m} shape={shape}", out_pq


2025-12-19 09:57:34,080 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.


In [10]:
from datetime import datetime

logs = []
for year in range(2020, 2025):
    for month in range(1, 13):
        try:
            msg, outp = download_era5_month(year, month, AREA, VARIABLES)
            logs.append((year, month, msg, str(outp)))
            print(year, month, msg)
        except Exception as e:
            logs.append((year, month, f"FAIL: {type(e).__name__}: {e}", ""))
            print("FAIL", year, month, e)

log_df = pd.DataFrame(logs, columns=["year","month","status","path"])
log_df.head(20)


2025-12-19 09:58:08,967 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2025-12-19 09:58:08,967 INFO Request ID is 09c7a636-4aeb-4d75-8b46-904c15a9fa4f
2025-12-19 09:58:09,280 INFO status has been updated to accepted
2025-12-19 09:58:19,548 INFO status has been updated to running
2025-12-19 10:02:47,756 INFO status has been updated to successful


2020 1 OK 202001 shape=(31248, 10)


2025-12-19 10:02:56,732 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2025-12-19 10:02:56,733 INFO Request ID is 799d383f-8676-4b42-998d-ea93132beca6
2025-12-19 10:02:57,064 INFO status has been updated to accepted
2025-12-19 10:03:19,717 INFO status has been updated to running
2025-12-19 10:07:33,850 INFO status has been updated to successful


2020 2 OK 202002 shape=(29232, 10)


2025-12-19 10:07:38,558 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2025-12-19 10:07:38,558 INFO Request ID is 6d29825b-7649-41e4-abea-6f704ee91910
2025-12-19 10:07:38,846 INFO status has been updated to accepted
2025-12-19 10:07:48,179 INFO status has been updated to running
2025-12-19 10:12:15,273 INFO status has been updated to successful


...

2024 1 OK 202401 shape=(31248, 10)


2025-12-19 14:16:04,826 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2025-12-19 14:16:04,827 INFO Request ID is 0b861e9b-3362-4d38-838a-8ceee954bd3d
2025-12-19 14:16:05,144 INFO status has been updated to accepted
2025-12-19 14:16:14,539 INFO status has been updated to running
2025-12-19 14:19:09,623 INFO status has been updated to successful


2024 2 OK 202402 shape=(29232, 10)


2025-12-19 14:19:16,550 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)


KeyboardInterrupt: 